# Store

> Module for constructing AppletStore.

In [ ]:
#| default_exp utils.store

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import requests
import yaml
import json
import os

from llmcam.core.oas_to_requests import *
from llmcam.core.fn_to_fc import *
from typing import Optional, Callable

## Dynamic API installation

In [ ]:
#| export
def load_oas(
    oas_url: str = "https://tie.digitraffic.fi/swagger/openapi.json",  # OpenAPI Specification URL
    destination: str = "api/road_digitraffic.json",  # Destination file
    overwrite: bool = False  # Overwrite existing file
) -> dict:  # OpenAPI Specification
    """Load OpenAPI Specification from URL or file."""
    # Create destination directory if it does not exist
    os.makedirs(os.path.dirname(destination), exist_ok=True)

    # Download OpenAPI Specification if it does not exist or overwrite is True
    if not os.path.exists(destination) or overwrite:
        r = requests.get(oas_url)
        with open(destination, "w") as f:
            f.write(r.text)

    # Load OpenAPI Specification
    with open(destination, "r") as f:
        if destination.endswith(".json"):
            return json.load(f)
        elif destination.endswith(".yaml") or destination.endswith(".yml"):
            return yaml.load(f)
        else:
            raise ValueError("Invalid file format")

In [ ]:
show_doc(load_oas)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/utils/store.py#L19){target="_blank" style="float:right; font-size:smaller"}

### load_oas

>      load_oas (oas_url:str='https://tie.digitraffic.fi/swagger/openapi.json',
>                destination:str='api/road_digitraffic.json',
>                overwrite:bool=False)

*Load OpenAPI Specification from URL or file.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| oas_url | str | https://tie.digitraffic.fi/swagger/openapi.json | OpenAPI Specification URL |
| destination | str | api/road_digitraffic.json | Destination file |
| overwrite | bool | False | Overwrite existing file |
| **Returns** | **dict** |  | **OpenAPI Specification** |

Usage to download three DigiTraffic endpoints

In [ ]:
road_digitraffic = load_oas(
    oas_url="https://tie.digitraffic.fi/swagger/openapi.json",
    destination="api/road_digitraffic.json",
    overwrite=False
)
train_digitraffic = load_oas(
    oas_url="https://rata.digitraffic.fi/swagger/openapi.json",
    destination="api/train_digitraffic.json",
    overwrite=False
)
marine_digitraffic = load_oas(
    oas_url="https://meri.digitraffic.fi/swagger/openapi.json",
    destination="api/marine_digitraffic.json",
    overwrite=False
)

Test functions with GPT:

In [ ]:
#| eval: false
messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    ("user", "Get me information about the trains departing on 2017-11-09 with train number 1."),
])
complete(messages, api_schema("https://rata.digitraffic.fi", train_digitraffic, fixup=generate_request))
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
Get me information about the trains departing on 2017-11-09 with train number 1.
>> Assistant:
Here's the information about the train number 1 that departed on 2017-11-09:  - **Operator**: VR -
**Train Type**: IC (InterCity) - **Train Category**: Long-distance - **Running Currently**: No -
**Cancelled**: No - **Timetable Type**: Regular - **Timetable Acceptance Date**: 2017-07-21  ###
Timetable:  1. **Departure - Helsinki (HKI)**    - Scheduled Time: 05:28    - Actual Time: 05:30:33
- Delay: 3 minutes    - Departure Track: 7  2. **Arrival - Pasila (PSL)**    - Scheduled Time: 05:33
- Actual Time: 05:35:26    - Delay: 2 minutes    - Arrival Track: 4  3. **Departure - Pasila (PSL)**
- Scheduled Time: 05:34    - Actual Time: 05:37:13    - Delay: 3 minutes  4. **Arrival - Tikkurila
(TKL)**    - Scheduled Time: 05:43    - Actual Time: 05:45:29    - Delay: 2 minutes    - Arrival
Track: 3  5. 

In [ ]:
#| eval: false
messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    ("user", "Get me information about the trains departing on 2017-11-09 with train number 1.")
])
complete(messages, api_schema("https://rata.digitraffic.fi", train_digitraffic, fixup=generate_request))
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
Get me information about the trains departing on 2017-11-09 with train number 1.
>> Assistant:
Train number 1, departing on November 9, 2017, is an InterCity (IC) type categorized as a long-
distance train and operated by VR (operator code 10).  Here's a summary of its schedule and run:  -
**Departure from Helsinki (HKI)**: Scheduled at 07:28 AM, actual departure at 07:30 AM. There was a
3-minute delay. - **Arrival and Departure at Pasila (PSL)**:     - Arrival: Scheduled at 07:33 AM,
actual arrival at 07:35 AM, with a 2-minute delay.   - Departure: Scheduled at 07:34 AM, actual
departure at 07:37 AM, with a 3-minute delay. - **Arrival at Joensuu (JNS)**: Scheduled at 11:45 AM,
actual arrival at 11:52 AM. The train had an 8-minute delay. Causes of this delay include traffic
congestion.  The train was not cancelled, ran according to a regular timetable, and had several
stops along the ro

In [ ]:
#| eval: false
messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    ("user", "Are there any active nautical warnings?")
])
complete(messages, api_schema("https://meri.digitraffic.fi", marine_digitraffic, fixup=generate_request))
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
Are there any active nautical warnings?
>> Assistant:
There are several active nautical warnings:  1. **VEITSILUOTO FAIRWAY**: The leading light
Ropposenkari Front (No. 9151) is out of order at position 65-38.21N 024-37.77E.  2. **OULU
FAIRWAY**: The leading line Pensaskari Rear (No. 8986) is out of order at position 65-03.24N
025-16.05E.  3. **HAILUOTO FIXED LINK**: The construction site will close the bridge openings during
the winter season. They are expected to open by April 30, 2025.  4. **FINNISH SEA AREAS AND INLAND
WATERWAYS**: Due to winter conditions, aids to navigation may be out of position or unlit; vessels
are advised to navigate with caution.  5. **TURKU HARBOUR**: Construction works at berths S1 – 24,
vessels are advised to navigate with caution and contact Archipelago VTS for more information.  6.
**LAPPOHJA FAIRWAY**: A buoy is adrift towards the shoreline, precaution 

Test with Pet Store endpoint:

In [ ]:
#| eval: false
pet_store = load_oas(
    oas_url="https://petstore3.swagger.io/api/v3/openapi.json", 
    destination="api/petstore3.json"
)

messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    ("user", "Find me some available pets.")
])
complete(messages, api_schema("https://petstore3.swagger.io/api/v3", pet_store, fixup=generate_request))
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
Find me some available pets.
>> Assistant:
I'm sorry, but it seems there was an error processing your request to find available pets. Please
try again later or check back for any updates.


In [ ]:
#| export
def add_api_tools(
    tools: list,  # List of existing tools
    service_name: str,  # Name of the API service
    base_url: str,  # Base URL of the API service
    oas_url: Optional[str] = None,  # OpenAPI Specification URL
    oas_destination: Optional[str] = None # OpenAPI Specification destination file
):
    """Add API tools to the toolbox."""
    # Load OpenAPI Specification
    if oas_url is None:
        oas_url = f"{base_url}/swagger/openapi.json"
    if oas_destination is None:
        oas_destination = f"api/{service_name}.json"
    oas = load_oas(oas_url, oas_destination, overwrite=True)

    # Create tool schema and append to toolbox
    schema = api_schema(base_url, oas, service_name=service_name, fixup=generate_request)
    tools.extend(schema)

In [ ]:
show_doc(add_api_tools)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/utils/store.py#L44){target="_blank" style="float:right; font-size:smaller"}

### add_api_tools

>      add_api_tools (tools:list, service_name:str, base_url:str,
>                     oas_url:Optional[str]=None,
>                     oas_destination:Optional[str]=None)

*Add API tools to the toolbox.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| tools | list |  | List of existing tools |
| service_name | str |  | Name of the API service |
| base_url | str |  | Base URL of the API service |
| oas_url | Optional | None | OpenAPI Specification URL |
| oas_destination | Optional | None | OpenAPI Specification destination file |

Example usage:

In [ ]:
tools = []
add_api_tools(tools, "road_digitraffic", "https://tie.digitraffic.fi")
assert len(tools) > 0
tools[0]

{'type': 'function',
 'function': {'name': 'tmsStationsDatex2Xml',
  'description': 'The static information of TMS stations in Datex2 format (Traffic Measurement System / LAM)',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'object',
     'properties': {'state': {'type': 'string',
       'description': 'Road station state',
       'default': 'ACTIVE',
       'enum': ['ALL', 'REMOVED', 'ACTIVE']}},
     'required': []}},
   'required': []},
  'metadata': {'url': 'https://tie.digitraffic.fi/api/beta/tms-stations-datex2.xml',
   'method': 'get',
   'accepted_queries': ['state'],
   'service': 'road_digitraffic'},
  'fixup': 'llmcam.core.oas_to_requests.generate_request'}}

In [ ]:
#| eval: false
messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    ("user", "Get the weather camera information for the stations with ID C01503 and C01504."),
])
complete(messages, tools=tools)
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
Get the weather camera information for the stations with ID C01503 and C01504.
>> Assistant:
Here is the weather camera information for the stations with ID C01503 and C01504:  ### Station ID:
C01503 - **Name:** Road 51 Inkoo - **Camera Type:** BOSCH - **Coordinates:** Latitude 60.05374,
Longitude 23.99616 - **Municipality:** Inkoo - **Province:** Uusimaa - **Nearest Weather Station
ID:** 1013 - **Collection Status:** Gathering - **Collection Interval:** 600 seconds - **Presets:**
- **Inkooseen:**     - Resolution: 1280x720     - [Image
URL](https://weathercam.digitraffic.fi/C0150301.jpg)   - **Hankoon:**     - Resolution: 1280x720
- [Image URL](https://weathercam.digitraffic.fi/C0150302.jpg)   - **Tienpinta (Special Direction):**
- Resolution: 1280x720     - [Image URL](https://weathercam.digitraffic.fi/C0150309.jpg)  ###
Station ID: C01504 - **Name:** Road 2 Karkkila, Kappeli - **Came

## Dynamic functions from installed libraries

It is possible to dynamically import a function in installed library and execute it with `importlib`.

In [ ]:
#| export
from importlib import import_module
from typing import Callable, Any
from llmcam.core.fn_to_fc import tool_schema

Utility to add functions from installed libraries to ToolBox.

In [ ]:
#| export
def add_function_tools(
    tools: list,  # List of existing tools
    service_name: str,  # Name of the service
    function_names: list[str],  # List of function names (with module prefix)
):
    """Add function tools to the toolbox."""
    # Import functions
    for function_name in function_names:
        # Get module prefix
        module_prefix = function_name.split(".")
        if len(module_prefix) == 1:
            module_prefix = "builtins"
        else:
            module_prefix = ".".join(module_prefix[:-1])

        # Get function name without module prefix
        func_name = function_name.split(".")[-1]

        # Import function
        if module_prefix == "builtins":
            func: Callable = getattr(__builtins__, func_name, None)
        else:
            func: Callable = getattr(import_module(module_prefix), func_name, None)

        # Raise error if function not found
        if func is None:
            raise ValueError(f"Function not found: {function_name}")
        
        # Create tool schema and append to toolbox
        tools.append(tool_schema(func=func, service_name=service_name))

In [ ]:
show_doc(add_function_tools)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/utils/store.py#L69){target="_blank" style="float:right; font-size:smaller"}

### add_function_tools

>      add_function_tools (tools:list, service_name:str,
>                          function_names:list[str])

*Add function tools to the toolbox.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| tools | list | List of existing tools |
| service_name | str | Name of the service |
| function_names | list | List of function names (with module prefix) |

Example usage:

In [ ]:
tools = []
add_function_tools(
    tools, 
    "youtube_live", 
    [
        "llmcam.core.fn_to_fc.capture_youtube_live_frame_and_save", 
        "llmcam.core.fn_to_fc.ask_gpt4v_about_image_file"
    ])
assert len(tools) == 2
tools

[{'type': 'function',
  'function': {'name': 'capture_youtube_live_frame_and_save',
   'description': 'Capture a jpeg file from YouTube Live and save in data directory',
   'parameters': {'type': 'object',
    'properties': {'link': {'anyOf': [{'type': 'string',
        'description': 'YouTube Live link'},
       {'type': 'null',
        'description': 'A default value will be automatically used.'}]},
     'place': {'anyOf': [{'type': 'string',
        'description': 'Location of live image'},
       {'type': 'null',
        'description': 'A default value will be automatically used.'}]}},
    'required': []},
   'metadata': {'module': 'llmcam.core.fn_to_fc', 'service': 'youtube_live'}}},
 {'type': 'function',
  'function': {'name': 'ask_gpt4v_about_image_file',
   'description': 'Tell all about quantitative information from a given image file',
   'parameters': {'type': 'object',
    'properties': {'path': {'type': 'string',
      'description': 'Path to the image file'}},
    'requir

In [ ]:
#| eval: false
messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to assist the user."),
    ("user", "Hi, can you capture and extract information from a YouTube Live? Use the default link.")
])
complete(messages, tools=tools)
print_msgs(messages)

[youtube] Extracting URL: https://www.youtube.com/watch?v=LMZQ7eFhm58
[youtube] LMZQ7eFhm58: Downloading webpage
[youtube] LMZQ7eFhm58: Downloading ios player API JSON
[youtube] LMZQ7eFhm58: Downloading mweb player API JSON
[youtube] LMZQ7eFhm58: Downloading m3u8 information
[youtube] LMZQ7eFhm58: Downloading m3u8 information
09.12.2024 23:08:52 Olympiaterminaali
>> System:
You are a helpful system administrator. Use the supplied tools to assist the user.
>> User:
Hi, can you capture and extract information from a YouTube Live? Use the default link.
>> Assistant:
I've captured a frame from the YouTube Live stream and extracted the following information from the
image:  - **Timestamp:** 2024-12-09T23:08:52 - **Location:** Olympiaterminaali - **Image
Dimensions:** 1280 x 720 - **Buildings:** 5 buildings, ranging from 2-4 stories in height -
**Vehicles:** 15 cars, 1 truck, 2 boats - **Parking:** 10 available parking spaces - **Street
Lights:** 15 - **Persons:** 0 detected - **Time of Day:

## ToolBox management as FC

While the base functions for adding functions and APIs as tools work for appending new functions to an existing Python list, these functions cannot be easily transformed into tools to be used for function calling because:

- Changes must be made into the top-level `tools` and continue to be applied in all subsequent conversations.
- `tools` cannot be safely interpreted as a part of message as in some cases new tool changes may exceed token limit.

One solution to this is to introduce a special fix-up function is used for handlers which accesses the correct instance of `tools`. This function is to be created and attached to the `tools` instance at the moment of creation. However, this may cause issues on the application side as there might be multiple instances of `tools` being created. To combat this issue, we can have an alternative workflow involving a mapping of `session_tools` which maps each instance of `tools` with a corresponding unique ID. Paired with this mapping will be an fix-up function which extracts the tool instance and pass into core execution function.

In [ ]:
#| export

# Additional functions
def remove_tools(
    tools: list,  # List of existing tools
    service_name: str  # Name of the service
):
    """Remove tools from the toolbox."""
    tools[:] = [tool for tool in tools if ("service" not in tool["function"]["metadata"] or \
                                       tool["function"]["metadata"]["service"] != service_name)]

In [ ]:
#| export
import importlib

In [ ]:
#| export
def execute_handler_core(
    tools: list, # Tools for each session
    function_name: str,  # Name of the function to execute
    module: str,  # Module of the function
    **kwargs  # Keyword arguments
):
    """Execute the handler function by retrieving function with session ID."""
    # Get the function
    module = importlib.import_module(module)
    function = getattr(module, function_name, None)
    if function is None:
        raise ValueError(f"Function not found: {function_name}")
    
    if "service" in kwargs:
        del kwargs["service"]
    
    # Execute the function
    function(tools, **kwargs)

In [ ]:
#| export
def handler_schema(
    function: Callable,  # Handler function
    service_name: str = "toolbox_handler",  # Name of the service
    fixup: Optional[Callable] = None,  # Function to fixup function execution
    **kwargs  # Additional keyword arguments
):
    """Create a schema for handlers."""
    schema = tool_schema(function, service_name=service_name)
    schema["function"]["metadata"]["service"] = service_name

    # Add additional metadata
    for key, value in kwargs.items():
        schema["function"]["metadata"][key] = value

    if fixup: schema["function"]['fixup'] = f"{fixup.__module__}.{fixup.__name__}"

    # Remove tools from parameters
    if "tools" in schema["function"]["parameters"]["properties"]:
        del schema["function"]["parameters"]["properties"]["tools"]
        schema["function"]["parameters"]["required"].remove("tools")
    
    return schema

Example usage:

In [ ]:
tools = []
def execute_handler(function_name, **kwargs):
    execute_handler_core(tools, function_name, **kwargs)

tools.extend([handler_schema(function, service_name="toolbox_handler", fixup=execute_handler) for function in [
        add_api_tools,
        add_function_tools,
        remove_tools
    ]])
assert len(tools) == 3
tools[0]

{'type': 'function',
 'function': {'name': 'add_api_tools',
  'description': 'Add API tools to the toolbox.',
  'parameters': {'type': 'object',
   'properties': {'service_name': {'type': 'string',
     'description': 'Name of the API service'},
    'base_url': {'type': 'string',
     'description': 'Base URL of the API service'},
    'oas_url': {'anyOf': [{'type': 'string',
       'description': 'OpenAPI Specification URL'},
      {'type': 'null',
       'description': 'A default value will be automatically used.'}]},
    'oas_destination': {'anyOf': [{'type': 'string',
       'description': 'OpenAPI Specification destination file'},
      {'type': 'null',
       'description': 'A default value will be automatically used.'}]}},
   'required': ['service_name', 'base_url']},
  'metadata': {'module': '__main__', 'service': 'toolbox_handler'},
  'fixup': '__main__.execute_handler'}}

In [ ]:
execute_handler(
    "add_api_tools",
    module="__main__",
    service_name="road_digitraffic",
    base_url="https://tie.digitraffic.fi"
)
assert len(tools) > 3
tools[4]

{'type': 'function',
 'function': {'name': 'tmsStationsDatex2Json',
  'description': 'The static information of TMS stations in Datex2 format (Traffic Measurement System / LAM)',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'object',
     'properties': {'state': {'type': 'string',
       'description': 'Road station state',
       'default': 'ACTIVE',
       'enum': ['ALL', 'REMOVED', 'ACTIVE']}},
     'required': []}},
   'required': []},
  'metadata': {'url': 'https://tie.digitraffic.fi/api/beta/tms-stations-datex2.json',
   'method': 'get',
   'accepted_queries': ['state'],
   'service': 'road_digitraffic'},
  'fixup': 'llmcam.core.oas_to_requests.generate_request'}}

Simulated GPT workflow:

In [ ]:
from llmcam.core.fn_to_fc import complete, form_msg, form_msgs, print_msgs

In [ ]:
tools = []
def execute_handler(function_name, **kwargs):
    execute_handler_core(tools, function_name, **kwargs)

tools.extend([handler_schema(function, service_name="toolbox_handler", fixup=execute_handler) for function in [
        add_api_tools,
        add_function_tools,
        remove_tools
    ]])
assert len(tools) == 3

In [ ]:
#| eval: false
messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to assist the user."),
    ("user", "Add a new API service called 'road_digitraffic'. Use the base URL 'https://tie.digitraffic.fi', and the OpenAPI Specification URL 'https://tie.digitraffic.fi/swagger/openapi.json'.")
])
complete(messages, tools=tools)
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to assist the user.
>> User:
Add a new API service called 'road_digitraffic'. Use the base URL 'https://tie.digitraffic.fi', and
the OpenAPI Specification URL 'https://tie.digitraffic.fi/swagger/openapi.json'.
>> Assistant:
The API service 'road_digitraffic' has been successfully added with the base URL
'https://tie.digitraffic.fi' and the OpenAPI Specification URL
'https://tie.digitraffic.fi/swagger/openapi.json'.


In [ ]:
#| eval: false
len(tools)

65

In [ ]:
#| eval: false
messages.append(form_msg("user", "Get the weather camera information for the stations with ID C01503 and C01504."))
complete(messages, tools=tools)
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to assist the user.
>> User:
Add a new API service called 'road_digitraffic'. Use the base URL 'https://tie.digitraffic.fi', and
the OpenAPI Specification URL 'https://tie.digitraffic.fi/swagger/openapi.json'.
>> Assistant:
The API service 'road_digitraffic' has been successfully added with the base URL
'https://tie.digitraffic.fi' and the OpenAPI Specification URL
'https://tie.digitraffic.fi/swagger/openapi.json'.
>> User:
Get the weather camera information for the stations with ID C01503 and C01504.
>> Assistant:
Here is the weather camera information for the stations with ID C01503 and C01504:  ### Station ID:
C01503 - **Name:** kt51_Inkoo - **Camera Type:** BOSCH - **Location:** Latitude 60.05374, Longitude
23.99616 - **Nearest Weather Station ID:** 1013 - **Collection Status:** GATHERING -
**Municipality:** Inkoo - **Province:** Uusimaa - **Road:** Road 51, Section 14 - **Presets:**   -
**Inkooseen:**      -

In [ ]:
#| eval: false
messages.append(form_msg("user", "Remove the 'road_digitraffic' service."))
complete(messages, tools=tools)
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to assist the user.
>> User:
Add a new API service called 'road_digitraffic'. Use the base URL 'https://tie.digitraffic.fi', and
the OpenAPI Specification URL 'https://tie.digitraffic.fi/swagger/openapi.json'.
>> Assistant:
The API service 'road_digitraffic' has been successfully added with the base URL
'https://tie.digitraffic.fi' and the OpenAPI Specification URL
'https://tie.digitraffic.fi/swagger/openapi.json'.
>> User:
Get the weather camera information for the stations with ID C01503 and C01504.
>> Assistant:
Here is the weather camera information for the stations with ID C01503 and C01504:  ### Station ID:
C01503 - **Name:** kt51_Inkoo - **Camera Type:** BOSCH - **Location:** Latitude 60.05374, Longitude
23.99616 - **Nearest Weather Station ID:** 1013 - **Collection Status:** GATHERING -
**Municipality:** Inkoo - **Province:** Uusimaa - **Road:** Road 51, Section 14 - **Presets:**   -
**Inkooseen:**      -

In [ ]:
#| eval: false
# After removing the 'road_digitraffic' service, the tools should only contain the initial handlers
len(tools)

3

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()